In [73]:
## Header files 
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##For STrAdaBoost.R2
from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ## For two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
#import geopandas as gdp
#from matplotlib.colors import ListedColormap
#import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines
import folium
import glob

from statistics import mean

pd.options.display.max_columns = None

##Uploading the Instance Weighing Kernel Ridge Regression library
from IW_KRR import InstanceKRR


print("Done uploading repositories")

Done uploading repositories


In [81]:
########################################################################################################################################################
# ConcreteData_df = pd.read_excel('UCI_regression/Concrete_Data.xls') ## 'Cement' found to be correlated at 0.4 :: 100
# HousingData_df = pd.read_csv('UCI_regression/BostonHousing/BostonHousing.csv') ## 'nox' found to be correlated at 0.4 :: [0.385 - 0.871] :: 50
dropcol_initial = ['name']
AutoData_df = pd.read_csv('UCI_regression/MPG/Auto.csv') ## horsepower column has correlation 0.4 :: [46 - 230] :: 30
AutoData_df = AutoData_df.drop(dropcol_initial, axis = 1)
print("The shape of the Input data is: ", AutoData_df.shape)

############################# Preprocessing Data ################################################################
## Finding the correlation first
# print("The correlation matrix is: ")
# print(ConcreteData_df.corr())

# col = ConcreteData_df['Cement']
# ConcreteData_df = ConcreteData_df.sort_values(by=['Cement'])
# col = HousingData_df['nox']


# drop_col = ['Cement']
drop_col = ['horsepower']
# drop_col = ['nox']

# Train_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] <= 225)]
# Train_df = HousingData_df.loc[(HousingData_df['nox'] > 0.475) & (HousingData_df['nox'] <= 0.600)]
Train_df = AutoData_df.loc[(AutoData_df['horsepower'] > 80) & (AutoData_df['horsepower'] <= 110)]
Train_df = Train_df.drop(drop_col, axis = 1)
Train_df = Train_df.reset_index(drop=True)
print(Train_df.shape)

# Source_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 225) & (ConcreteData_df['Cement'] <= 350)]
# Source_df = HousingData_df.loc[(HousingData_df['nox'] <= 0.475)]
Source_df = AutoData_df.loc[(AutoData_df['horsepower'] <= 80)]
Source_df = Source_df.drop(drop_col, axis = 1)
Source_df = Source_df.reset_index(drop=True)
print(Source_df.shape)

# Test_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 350)]
# Test_df = HousingData_df.loc[(HousingData_df['nox'] > 0.600)]
Test_df = AutoData_df.loc[(AutoData_df['horsepower'] > 110)]
Test_df = Test_df.drop(drop_col, axis = 1)
Test_df = Test_df.reset_index(drop=True)
print(Test_df.shape)

The shape of the Input data is:  (392, 8)
(157, 7)
(119, 7)
(116, 7)


In [82]:
# target_column = ['ConcreteCompressiveStrength']
# target_column = ['medv']
target_column = ['mpg']

# Train_df = pd.concat([Train_df, Source_df], ignore_index=True)

Train_df_y = Train_df[target_column]
Train_df_X = Train_df.drop(target_column, axis = 1)

Test_df_y = Test_df[target_column]
Test_df_X = Test_df.drop(target_column, axis = 1)

Source_df_y = Source_df[target_column]
Source_df_X = Source_df.drop(target_column, axis = 1)

print(Train_df_X.shape)
print(Test_df_X.shape)
print(Source_df_X.shape)

##Converting to numpy arrays

np_train_X = Train_df_X.to_numpy()
np_train_y = Train_df_y.to_numpy()

np_test_X = Test_df_X.to_numpy()
np_test_y = Test_df_y.to_numpy()

np_source_X = Source_df_X.to_numpy()
np_source_y = Source_df_y.to_numpy()

## Convert the testing array into a 1D list
np_test_y_list = np_test_y.ravel()


(157, 6)
(116, 6)
(119, 6)


In [83]:
## Fit the data
krr = InstanceKRR(lmbd = 0.5)
krr.fit(Source_df_X,Source_df_y)

In [84]:
yhat = krr.kRR_predict(Test_df_X)

## Find alpha weights
alpha_obj = krr.Solve_alpha(Train_df_X, Train_df_y)

print(alpha_obj.X_aux_)
print("\n\n")
print(np_train_X)

## Make the prediction
y_prd = krr.predict(Test_df_X)
# len(y_prd), len(np_test_y_list)


[[4.000e+00 1.130e+02 2.372e+03 1.500e+01 7.000e+01 3.000e+00]
 [6.000e+00 1.980e+02 2.833e+03 1.550e+01 7.000e+01 1.000e+00]
 [6.000e+00 1.990e+02 2.774e+03 1.550e+01 7.000e+01 1.000e+00]
 [6.000e+00 2.000e+02 2.587e+03 1.600e+01 7.000e+01 1.000e+00]
 [4.000e+00 9.700e+01 2.130e+03 1.450e+01 7.000e+01 3.000e+00]
 [4.000e+00 1.100e+02 2.672e+03 1.750e+01 7.000e+01 2.000e+00]
 [4.000e+00 1.070e+02 2.430e+03 1.450e+01 7.000e+01 2.000e+00]
 [4.000e+00 1.040e+02 2.375e+03 1.750e+01 7.000e+01 2.000e+00]
 [6.000e+00 1.990e+02 2.648e+03 1.500e+01 7.000e+01 1.000e+00]
 [4.000e+00 9.700e+01 2.130e+03 1.450e+01 7.100e+01 3.000e+00]
 [4.000e+00 1.400e+02 2.264e+03 1.550e+01 7.100e+01 1.000e+00]
 [4.000e+00 1.130e+02 2.228e+03 1.400e+01 7.100e+01 3.000e+00]
 [6.000e+00 2.320e+02 2.634e+03 1.300e+01 7.100e+01 1.000e+00]
 [6.000e+00 2.250e+02 3.439e+03 1.550e+01 7.100e+01 1.000e+00]
 [6.000e+00 2.500e+02 3.329e+03 1.550e+01 7.100e+01 1.000e+00]
 [6.000e+00 2.500e+02 3.302e+03 1.550e+01 7.100e+01 1.0

In [85]:
## Convert the predicted array into a 1D list
y_prd = y_prd.ravel()
#y_prd

In [86]:
r2_score_IWKRR_values = pearsonr(np_test_y_list, y_prd)
r2_score_IWKRR = (r2_score_IWKRR_values[0])**2
print("R^2 of IWKRR:", r2_score_IWKRR)

R^2 of IWKRR: 0.062423126477885046


In [87]:
mse_IWKRR = np.sqrt(mean_squared_error(np_test_y_list, y_prd))
print("RMSE of IWKRR:", mse_IWKRR)



RMSE of IWKRR: 16.249421614813926
